In [1]:
!pip install tflearn
import os
import cv2
import time
import random
import numpy as np
from random import shuffle

import tensorflow as tf
# from tensorflow.keras.applications.inception_v3 import preprocess_input

import seaborn as sns
import matplotlib.pyplot as plt
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation
tf.__version__, np.__version__

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 8.3 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=43e9af02e0f3aa29085bd859373cbd07cffd04c67d516907fbe765e83e59f5bb
  Stored in directory: /root/.cache/pip/wheels/65/9b/15/cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn


Instructions for updating:
non-resource variables are not supported in the long term


('2.9.2', '1.21.6')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip "/content/gdrive/MyDrive/Colab Notebooks/CV_2023_SC_Dataset.zip"


In [ ]:
# Create folder


In [4]:
def create_label(image_name):
  if image_name.__contains__('A'):
      return np.array([1, 0, 0, 0, 0])
  elif image_name.__contains__('B'):
      return np.array([0, 1, 0, 0, 0])
  elif image_name.__contains__('C'):
      return np.array([0, 0, 1, 0, 0])
  elif image_name.__contains__('D'):
      return np.array([0, 0, 0, 1, 0])
  elif image_name.__contains__('E'):
      return np.array([0, 0, 0, 0, 1])

In [5]:
DataPath = "/content/Data"

IMG_SIZE = 200

def extractData(path):
    train_images = []
    test_images = []

    Personsfolder = os.listdir(path)
    for person in Personsfolder:
        personPath = os.path.join(path, person)
        train_testOfPerson = os.listdir(personPath)

        if train_testOfPerson[1] == 'Train':
            trainPath = os.path.join(personPath, train_testOfPerson[1])
            trainImgNames = os.listdir(trainPath)
            for i in trainImgNames:
                if 'SigVerificationTrainLabels' not in i:
                    img = cv2.imread(os.path.join(trainPath, i), 0)
                    img_data = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    img_normalized = img_data / 255.0

                    train_images.append([np.array(img_normalized), create_label(i)])

        if train_testOfPerson[0] == 'Test':
            testPath = os.path.join(personPath, train_testOfPerson[0])
            testImgNames = os.listdir(testPath)
            for i in testImgNames:
                if 'SigVerificationTestLabels' not in i:
                    img = cv2.imread(os.path.join(testPath, i), 0)
                    img_data = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
                    img_normalized = img_data / 255.0

                    test_images.append([np.array(img_normalized), create_label(i)])
    shuffle(train_images)
    shuffle(test_images)
    return train_images, test_images










In [6]:
train, test = extractData(DataPath)

In [7]:
# IMG_SIZE = 50
X_train = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array([i[1] for i in train])

X_test = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array([i[1] for i in test])


In [8]:
# IMG_SIZE = 50

# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)
# Model structure

network = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 5, activation='softmax')
network = regression(network, optimizer='adam', loss='categorical_crossentropy', learning_rate=0.001)

model = tflearn.DNN(network, tensorboard_verbose=3)
if (os.path.exists('model.tfl.meta')):
  model.load('./model.tfl')
else:
  model.fit(X_train, y_train, n_epoch=10,show_metric=True, validation_set=0.2)
  model.save('model.tfl')

predictions = model.predict(X_test)


Training Step: 29  | total loss: 0.20406 | time: 27.644s
| Adam | epoch: 010 | loss: 0.20406 - acc: 0.9121 -- iter: 128/160
Training Step: 30  | total loss: 0.21767 | time: 44.200s
| Adam | epoch: 010 | loss: 0.21767 - acc: 0.9181 | val_loss: 0.06057 - val_acc: 1.0000 -- iter: 160/160
--


In [9]:
predictions = model.predict(X_test)

In [10]:
print(predictions)

[[9.99871016e-01 1.51817803e-09 6.36135444e-19 6.01114417e-28
  1.29118751e-04]
 [2.29773080e-12 3.13198604e-13 7.35429421e-16 9.99999940e-01
  4.85789522e-21]
 [9.99999940e-01 1.32670472e-10 2.37760783e-18 1.46315098e-30
  3.34546236e-12]
 [5.96583822e-21 1.44902204e-20 7.47538730e-22 9.99999940e-01
  1.43249451e-29]
 [2.27255610e-11 1.80904649e-03 1.82121766e-12 1.99491329e-19
  9.98190939e-01]
 [1.32710852e-07 9.99998391e-01 5.01666939e-08 8.49732350e-12
  1.42773024e-06]
 [1.86628782e-10 2.11896733e-07 3.00689160e-21 4.92870470e-32
  9.99999702e-01]
 [8.33997110e-14 5.42769172e-13 2.23665204e-17 9.99999940e-01
  1.34828319e-22]
 [7.81349740e-19 5.53104507e-11 9.99999940e-01 1.04434289e-12
  7.57307270e-19]
 [5.30418638e-14 1.13050856e-01 8.86949122e-01 2.65945333e-13
  3.65953490e-10]
 [2.88538460e-22 8.96701741e-24 2.49995171e-22 9.99999940e-01
  2.75210800e-30]
 [1.33223857e-16 1.02051324e-07 9.99999821e-01 6.84598829e-17
  2.90760206e-16]
 [2.43681313e-07 1.55330461e-04 2.155656

In [11]:
pred = []
for prediction in predictions:
    max_val = np.argmax(prediction)
    pred.append(max_val)
print((pred))

act = []
for prediction in y_test:
    max_val = np.argmax(prediction)
    act.append(max_val)
print((act))

[0, 3, 0, 3, 4, 1, 4, 3, 2, 2, 3, 2, 4, 4, 2, 2, 1, 1, 3, 1, 4, 0, 0, 3, 1, 0, 4, 0, 2, 2, 1, 1, 2, 3, 3, 0, 0, 4, 2, 4]
[0, 3, 0, 3, 4, 1, 4, 3, 2, 1, 3, 2, 4, 4, 2, 2, 1, 1, 3, 1, 4, 0, 0, 3, 1, 0, 4, 0, 2, 2, 1, 1, 2, 3, 3, 0, 0, 4, 2, 4]


In [12]:
c = 0
for i in range(len(pred)):
  if pred[i] == act[i]:
    c += 1
print("Test acc = " + str((c/len(pred)*100)) +"%")

Test acc = 97.5%
